In [5]:
using PrettyTables
using CSV
using DataFrames
using JLD2
using MAT
using Plots
using Profile
using PProf
include("quickscore_preparation.jl")
include("quickscore_algorithm.jl")

file = jldopen("variables/m_19.jld", "r")
previn = file["previn"]
pfminneg = file["pfminneg"]
pfmin = file["pfmin"]
close(file)

# pfplus,posterior,dt = 0,0,0
pfplus, posterior, dt = quickscore(previn,pfmin,pfminneg,true)

# m, n = !isempty(pfmin) ? size(pfmin) : (0, length(previn)) 
# prev = repeat(previn',inner=(n+1,1)) 
# for i in 1:n prev[i+1,i]=1 end 
# prevend = vcat(1,previn) 
# prevminneg = !isempty(pfminneg) ? prev.*pfminneg' : prev.*ones(n+1,n) 

# v = rand([0,1],15)
# myset = findall(v.==1)

# exp.(sum(log.(1e-50 .+ (prod(pfmin[myset, :],dims=1) .* prevminneg .+ (1 .- prev))), dims=2))
# matrix = BigFloat.(1e-50 .+ (prod(pfmin[myset, :],dims=1) .* prevminneg .+ (1 .- prev)))
# prod(matrix,dims=2)

# prod(pfmin[myset, :],dims=1)


m=19
Binary
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


([1.3690144551831682, 0.05813270816142494, 0.41435450885376696, 0.07515348485334647, 0.03568474595232682, 0.3181945108799545, 0.06465152549800351, 0.10980253852817555, 0.12120018488765211, 0.18764417803629027  …  0.27256332440411307, 0.12051612013489248, 0.19542090686776398, 0.06695727976296702, 0.14387350507789523, 0.06683476049810413, 0.4977944677592737, 0.10628429539060562, 0.6756187442882688, 0.06572776389875398], [0.0008781859846702725, 0.011056158571440227, 0.0018778831826386062, 0.0006622134586413259, 0.007379495325726182, 0.0013527849026807135, 0.003122439199385806, 0.0032687631325266403, 0.004328600864160095, 0.005575642620276375  …  0.006309332973850181, 0.001929180497871877, 0.003540995223829473, 0.0015245902189892015, 0.0030149615848566068, 0.0016109546619039138, 0.009484351022729532, 0.003059375107225095, 0.016129379102612203, 0.0011052419830163873], 27.11491298675537)

In [12]:
# @code_warntype quickscore(previn, pfmin, pfminneg, true)

# typeof(true)

Bool